# Criar Arquivos CSV para Retorno Diário e Mensal

In [101]:
import pandas as pd
import zipfile 
import requests

pd.options.display.float_format = '{:4f}'.format #para melhor visualização dos dados (casas decimais muito grandes)

# jan/21 até ago/24
anos = range(2021, 2025)  
meses = range(1, 13)  

anos_meses = [(ano, f"{mes:02d}") for ano in anos for mes in meses if (ano == 2024 and mes <= 9) or (ano < 2024)]

# Usar Nome associado ao CNPJ:

cadastro_fundos = pd.read_csv('https://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv',
                              sep = ';',
                              encoding = 'ISO-8859-1')

cadastro_fundos = cadastro_fundos[['CNPJ_FUNDO', 'DENOM_SOCIAL']]

cadastro_fundos = cadastro_fundos.drop_duplicates()

base_dados = pd.DataFrame()  # DF vazio 

for ano, mes in anos_meses:

    url = f"https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_{ano}{mes}.zip"

    paginaweb = requests.get(url)

    with open(f"inf_diario_fi_{ano}{mes}.zip", 'wb') as arquivo_cvm:

        arquivo_cvm.write(paginaweb.content)

    arquivo_zip = zipfile.ZipFile(f"inf_diario_fi_{ano}{mes}.zip")

    dados_fundos = pd.read_csv(arquivo_zip.open(arquivo_zip.namelist()[0]), sep=';', encoding='ISO-8859-1')
    
    # processar dados ordenados cronologicamente
    data_inicio_mes = (dados_fundos['DT_COMPTC'].sort_values(ascending=True).unique())[0] 

    data_final_mes = (dados_fundos['DT_COMPTC'].sort_values(ascending=True).unique())[-1]

    dados_fundos_org_dt = dados_fundos[(dados_fundos['DT_COMPTC'].isin([data_inicio_mes, data_final_mes]))]  # your processing code here
    
    dados_fundos_org_dt = pd.merge(dados_fundos_org_dt, cadastro_fundos, how="left", left_on="CNPJ_FUNDO", right_on="CNPJ_FUNDO")
    
    dados_fundos_org_dt = dados_fundos_org_dt[['CNPJ_FUNDO', 'DENOM_SOCIAL', 'DT_COMPTC', 'VL_QUOTA', 'VL_PATRIM_LIQ', 'NR_COTST']]
    
    # filtrando o fundo escolhido (SICREDI FIA PETROBRAS)
    fundo_sicredi_petr = dados_fundos_org_dt[dados_fundos_org_dt['CNPJ_FUNDO'].str.contains('08.336.054/0001-34', na=False)]
    
    # Concatenando dados em um dataframe (base de dados)
    base_dados = pd.concat([base_dados, fundo_sicredi_petr], ignore_index=True)

base_dados.to_csv('Dados_Inicio_Fim_Mes_SICREDIPETR.csv', index=False)

C:\Users\Guima\AppData\Local\Temp\ipykernel_11312\4171290958.py:15: DtypeWarning:

Columns (14,17,18,20,22,24,27,37,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.



In [1]:
# dataframe com variação diária
import pandas as pd
import zipfile 
import requests

pd.options.display.float_format = '{:4f}'.format #para melhor visualização dos dados (casas decimais muito grandes)

# jan/21 até ago/24
anos = range(2021, 2025)  
meses = range(1, 13)  

anos_meses = [(ano, f"{mes:02d}") for ano in anos for mes in meses if (ano == 2024 and mes <= 9) or (ano < 2024)]

# Usar Nome associado ao CNPJ:

cadastro_fundos = pd.read_csv('https://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv',
                              sep = ';',
                              encoding = 'ISO-8859-1')

cadastro_fundos = cadastro_fundos[['CNPJ_FUNDO', 'DENOM_SOCIAL']]

cadastro_fundos = cadastro_fundos.drop_duplicates()

base_dados2 = pd.DataFrame()  # DF vazio 

for ano, mes in anos_meses:

    url = f"https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_{ano}{mes}.zip"

    paginaweb = requests.get(url)

    with open(f"inf_diario_fi_{ano}{mes}.zip", 'wb') as arquivo_cvm:

        arquivo_cvm.write(paginaweb.content)

    arquivo_zip = zipfile.ZipFile(f"inf_diario_fi_{ano}{mes}.zip")

    dados_fundos = pd.read_csv(arquivo_zip.open(arquivo_zip.namelist()[0]), sep=';',    )

    # processar dados ordenados cronologicamente
    dados_fundos['DT_COMPTC'].sort_values(ascending=True)

    dados_fundos = pd.merge(dados_fundos, cadastro_fundos, how="left", left_on="CNPJ_FUNDO", right_on="CNPJ_FUNDO")

    dados_fundos = dados_fundos[['CNPJ_FUNDO', 'DENOM_SOCIAL', 'DT_COMPTC', 'VL_QUOTA', 'VL_PATRIM_LIQ', 'NR_COTST']]

    # fundo SICREDI
    fundo_sicredi_petr = dados_fundos[dados_fundos['CNPJ_FUNDO'].str.contains('08.336.054/0001-34', na=False)]

    # Concatenando dados em um dataframe (base de dados)
    base_dados2 = pd.concat([base_dados2, fundo_sicredi_petr], ignore_index=True)

base_dados2.to_csv('Dados_Inicio_Fim_Dia_SICREDIPETR.csv', index=False)
    

C:\Users\Guima\AppData\Local\Temp\ipykernel_19816\394475044.py:16: DtypeWarning: Columns (14,17,18,20,22,24,27,37,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  cadastro_fundos = pd.read_csv('https://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv',


Portanto, `base_dados == Dados_Mes` e `base_dados2 == Dados_Dia`

# Teste

In [2]:
import pandas as pd
# OBTER RETORNOS DIARIOS

base_dados2 = pd.read_csv("C:\\Users\\Guima\\Desktop\\Códigos IC\\DadosFundos_SICREDI_FIA_PETR\\Dados_Inicio_Fim_Dia_SICREDIPETR.csv")
# Converter a coluna 'DT_COMPTC' para yyyy-mm-dd
base_dados2['DT_COMPTC'] = pd.to_datetime(base_dados2['DT_COMPTC'])

# Criar uma coluna com o mês e o ano para facilitar o agrupamento
base_dados2['Ano_Mes_Dia'] = base_dados2['DT_COMPTC'].dt.to_period('D')

# Filtrar último dia de cada mês
ultimos_dias2 = base_dados2.groupby('Ano_Mes_Dia').apply(lambda x: x.loc[x['DT_COMPTC'].idxmax()])

# Garantir dados ordenados cronologicamente
ultimos_dias2 = ultimos_dias2.sort_values('DT_COMPTC')

# Calcular o retorno mensal como a variação percentual entre o valor da cota de um mês e o próximo
ultimos_dias2['Retorno_Diario'] = ultimos_dias2['VL_QUOTA'].pct_change()

ultimos_dias2['Retorno_Diario(%)'] = ultimos_dias2['Retorno_Diario'] *100

# Final:
ultimos_dias2[['DT_COMPTC', 'VL_QUOTA', 'Retorno_Diario(%)']]

,DT_COMPTC,VL_QUOTA,Retorno_Diario(%)
Ano_Mes_Dia,,,
2021-01-04,2021-01-04,0.962805,NaN
2021-01-05,2021-01-05,1.000309,3.895380
2021-01-06,2021-01-06,1.001292,0.098180
2021-01-07,2021-01-07,1.032193,3.086144
2021-01-08,2021-01-08,1.036161,0.384463
...,...,...,...
2024-09-09,2024-09-09,3.603852,1.041723
2024-09-10,2024-09-10,3.546663,-1.586902
2024-09-11,2024-09-11,3.543006,-0.103111


In [3]:
import plotly.express 

ultimos_dias2['Ano_Mes_Dia'] = ultimos_dias2['Ano_Mes_Dia'].dt.to_timestamp()

fig = plotly.express.line(ultimos_dias2, x='Ano_Mes_Dia', y="Retorno_Diario(%)")
fig.show()

c:\Users\Guima\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  v = v.dt.to_pydatetime()


In [28]:
# Volatilidade 
import numpy as np
ultimos_dias2['Volatilidade_Diaria(%)'] = ultimos_dias2['Retorno_Diario(%)'].rolling(window=22).std()*np.sqrt(252)

import plotly.express 

#ultimos_dias2['Ano_Mes_Dia'] = ultimos_dias2['Ano_Mes_Dia'].dt.to_timestamp()

fig1 = plotly.express.line(ultimos_dias2, x='Ano_Mes_Dia', y="Volatilidade_Diaria(%)")
fig1.show()

c:\Users\Guima\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [29]:
ultimos_dias2[['Ano_Mes_Dia', 'DT_COMPTC', 'VL_QUOTA', 'Retorno_Diario(%)', 'Volatilidade_Diaria(%)']]


,Ano_Mes_Dia,DT_COMPTC,VL_QUOTA,Retorno_Diario(%),Volatilidade_Diaria(%)
Ano_Mes_Dia,,,,,
2021-01-04,2021-01-04,2021-01-04,0.962805,NaN,NaN
2021-01-05,2021-01-05,2021-01-05,1.000309,3.895380,NaN
2021-01-06,2021-01-06,2021-01-06,1.001292,0.098180,NaN
2021-01-07,2021-01-07,2021-01-07,1.032193,3.086144,NaN
2021-01-08,2021-01-08,2021-01-08,1.036161,0.384463,NaN
...,...,...,...,...,...
2024-09-09,2024-09-09,2024-09-09,3.603852,1.041723,29.030797
2024-09-10,2024-09-10,2024-09-10,3.546663,-1.586902,29.453469
2024-09-11,2024-09-11,2024-09-11,3.543006,-0.103111,28.624382


In [27]:
ultimos_dias2.tail(160)

,CNPJ_FUNDO,DENOM_SOCIAL,DT_COMPTC,VL_QUOTA,VL_PATRIM_LIQ,NR_COTST,Ano_Mes_Dia,Retorno_Diario,Retorno_Diario(%),Volatilidade_Diaria(%),Indice_Sharpe_Diario
Ano_Mes_Dia,,,,,,,,,,,
2024-01-29,08.336.054/0001-34,SICREDI - FUNDO DE INVESTIMENTO EM AÇÕES PETRO...,2024-01-29,3.384027,1.674043e+08,15384,2024-01-29,0.014727,1.472722,21.333344,0.070475
2024-01-30,08.336.054/0001-34,SICREDI - FUNDO DE INVESTIMENTO EM AÇÕES PETRO...,2024-01-30,3.363857,1.664458e+08,15432,2024-01-30,-0.005960,-0.596018,21.270368,-0.031024
2024-01-31,08.336.054/0001-34,SICREDI - FUNDO DE INVESTIMENTO EM AÇÕES PETRO...,2024-01-31,3.374316,1.678396e+08,15467,2024-01-31,0.003109,0.310929,21.264096,0.013291
2024-02-01,08.336.054/0001-34,SICREDI - FUNDO DE INVESTIMENTO EM AÇÕES PETRO...,2024-02-01,3.464596,1.732119e+08,15505,2024-02-01,0.026755,2.675481,21.652778,0.121866
2024-02-02,08.336.054/0001-34,SICREDI - FUNDO DE INVESTIMENTO EM AÇÕES PETRO...,2024-02-02,3.421034,1.718847e+08,15564,2024-02-02,-0.012573,-1.257344,21.780140,-0.063775
...,...,...,...,...,...,...,...,...,...,...,...
2024-09-09,08.336.054/0001-34,SICREDI - FUNDO DE INVESTIMENTO EM AÇÕES PETRO...,2024-09-09,3.603852,1.872468e+08,15944,2024-09-09,0.010417,1.041723,23.631134,0.034531
2024-09-10,08.336.054/0001-34,SICREDI - FUNDO DE INVESTIMENTO EM AÇÕES PETRO...,2024-09-10,3.546663,1.843800e+08,16010,2024-09-10,-0.015869,-1.586902,23.716345,-0.055212
2024-09-11,08.336.054/0001-34,SICREDI - FUNDO DE INVESTIMENTO EM AÇÕES PETRO...,2024-09-11,3.543006,1.840439e+08,16063,2024-09-11,-0.001031,-0.103111,23.710383,-0.004974


In [109]:
import pandas as pd
import zipfile 
import requests

# OBTER RETORNOS MENSAIS
base_dados = pd.read_csv("C:\\Users\\Guima\\Desktop\\Códigos IC\\DadosFundos_SICREDI_FIA_PETR\\Dados_Inicio_Fim_Mes_SICREDIPETR.csv")
# Converter a coluna 'DT_COMPTC' para yyyy-mm-dd
base_dados['DT_COMPTC'] = pd.to_datetime(base_dados['DT_COMPTC'])

# Criar uma coluna com o mês e o ano para facilitar o agrupamento
base_dados['Ano_Mes'] = base_dados['DT_COMPTC'].dt.to_period('M')

# Filtrar apenas os últimos dias de cada mês
ultimos_dias = base_dados.groupby('Ano_Mes').apply(lambda x: x.loc[x['DT_COMPTC'].idxmax()])

# Garantir dados ordenados cronologicamente
ultimos_dias = ultimos_dias.sort_values('DT_COMPTC')

# Calcular o retorno mensal como a variação percentual entre o valor da cota de um mês e o próximo
ultimos_dias['Retorno_Mensal'] = ultimos_dias['VL_QUOTA'].pct_change()

ultimos_dias['Retorno_Mensal(%)'] = ultimos_dias['Retorno_Mensal'] *100

# Final:
ultimos_dias[['DT_COMPTC', 'VL_QUOTA', 'Retorno_Mensal(%)']]

,DT_COMPTC,VL_QUOTA,Retorno_Mensal(%)
Ano_Mes,,,
2021-01,2021-01-29,0.888905,NaN
2021-02,2021-02-26,0.741134,-16.623991
2021-03,2021-03-31,0.802532,8.284431
2021-04,2021-04-30,0.812623,1.257370
2021-05,2021-05-31,0.923566,13.652492
2021-06,2021-06-30,1.010887,9.454761
2021-07,2021-07-30,0.924342,-8.561359
2021-08,2021-08-31,0.987360,6.817620
2021-09,2021-09-30,0.988641,0.129760


In [ ]:
##### Não Batem:
# 2024-03	2024-03	2024-03-01	3.349856	0.097194
# 2024-04	2024-04	2024-04-30	3.602126	7.530768

# 2022-02	2022-02	2022-02-01	1.326437	2.003250
# 2022-03	2022-03	2022-03-31	1.344275	1.344866


In [110]:
# Calculando o retorno mensal como fator multiplicativo (1 + retorno_mensal/100)
ultimos_dias['Retorno_Fator'] = 1 + ultimos_dias['Retorno_Mensal(%)'] / 100

# Extraindo o ano da coluna 'DT_COMPTC'
ultimos_dias['Ano'] = ultimos_dias['DT_COMPTC'].dt.year

# Dicionário para armazenar o retorno anual
retorno_anual = {}

for ano, group in ultimos_dias.groupby("Ano"):
    retorno_anual[ano] = group['Retorno_Fator'].cumprod().iloc[-1] -1

# Transformar o dicionário em DataFrame
retorno_anual_df = pd.DataFrame(list(retorno_anual.items()), columns=['Ano', 'Retorno_Anual'])

# Definir o índice como 'Ano' para tornar o DataFrame mais legível
retorno_anual_df.set_index('Ano', inplace=True)

# Retorno anual %
retorno_anual_df['Retorno_Anual(%)'] = retorno_anual_df['Retorno_Anual'] * 100

retorno_anual_df

# 2021 não bate por conta do dado de 2021/01
# 2024 nao bate por conta do dado de 2024/09

,Retorno_Anual,Retorno_Anual(%)
Ano,,
2021,0.287568,28.756771
2022,0.455961,45.596066
2023,0.870131,87.013072
2024,0.186771,18.677134


In [15]:
# Sharpe Ratio

# Taxa Livre de Risco no Brasil - SELIC
RF_Rate_Ano = 0.1040
Dias_Uteis = 252
RF_Rate_Dia = (1 + RF_Rate_Ano) ** (1 / Dias_Uteis) - 1

RF_Rate_Dia_percent = RF_Rate_Dia * 100

ultimos_dias2['Indice_Sharpe_Diario'] = (ultimos_dias2['Retorno_Diario(%)'] - RF_Rate_Dia_percent) / ultimos_dias2['Volatilidade_Diaria(%)']

ultimos_dias2[['Ano_Mes_Dia', 'DT_COMPTC', 'VL_QUOTA', 'Retorno_Diario(%)', 'Volatilidade_Diaria(%)', 'Indice_Sharpe_Diario']]

,Ano_Mes_Dia,DT_COMPTC,VL_QUOTA,Retorno_Diario(%),Volatilidade_Diaria(%),Indice_Sharpe_Diario
Ano_Mes_Dia,,,,,,
2021-01-04,2021-01-04,2021-01-04,0.962805,NaN,NaN,NaN
2021-01-05,2021-01-05,2021-01-05,1.000309,3.895380,NaN,NaN
2021-01-06,2021-01-06,2021-01-06,1.001292,0.098180,NaN,NaN
2021-01-07,2021-01-07,2021-01-07,1.032193,3.086144,NaN,NaN
2021-01-08,2021-01-08,2021-01-08,1.036161,0.384463,NaN,NaN
...,...,...,...,...,...,...
2024-09-09,2024-09-09,2024-09-09,3.603852,1.041723,29.030797,0.034531
2024-09-10,2024-09-10,2024-09-10,3.546663,-1.586902,29.453469,-0.055212
2024-09-11,2024-09-11,2024-09-11,3.543006,-0.103111,28.624382,-0.004974
